# Athena audits: demo and state results
Show usage of the Audit class and demo on 2016 presidential contest in selected states

## Setup and define some utilities

In [1]:
from athena.audit import Audit
import math
import json
import sys

In [2]:
def make_audit(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule):
    "Convenience function to mak Audit class with given election parameters"

    election = {
        "alpha": alpha,
        "delta": delta,
        "candidates": candidates,
        "results": results,
        "ballots_cast": ballots_cast,
        "winners": winners,
        "name": name,
        "model": model,
        "pstop_goal": pstop_goal,
        "round_schedule": round_schedule,
    }
    a = Audit(audit_type, election['alpha'], election['delta'])
    a.add_election(election)
    return a

In [3]:
def find_next_round_size(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule):
    "Convenience function to call Audit class with given election parameters"

    election = {
        "alpha": alpha,
        "delta": delta,
        "candidates": candidates,
        "results": results,
        "ballots_cast": ballots_cast,
        "winners": winners,
        "name": name,
        "model": model,
        "pstop_goal": pstop_goal,
        "round_schedule": round_schedule,
    }
    a = Audit(audit_type, election['alpha'], election['delta'])
    a.add_election(election)
    x = a.find_next_round_size(election['pstop_goal'])
    return x

In [4]:
def sample90(margin, audit_type="ATHENA"):
    "Return sample size and other output given margin"

    assert 0.0 < margin < 1.0
    ballots_cast = 10000
    margin_votes = round(margin * ballots_cast)
    b = ballots_cast//2 - margin_votes // 2
    a = b + int(margin_votes)
    x = find_next_round_size(audit_type, 0.1, 1.0, ["A", "B"], [a, b], ballots_cast, 1, "state", "bin", [0.9], [])
    return (x['detailed']['A-B']['next_round_sizes'][0], x)

In [5]:
def sample90v(a, b, ballots_cast, audit_type="ATHENA"):
    "Return sample size etc. given votes for each of top two candidates in 1-winner contest"

    winner = max(a,b)
    loser = min(a,b)
    assert 0 < loser < winner < ballots_cast
    x = find_next_round_size(audit_type, 0.1, 1.0, ["A", "B"], [winner, loser], ballots_cast, 1, "state", "bin", [0.9], [])
    return (x['detailed']['A-B']['next_round_sizes'][0], x)

In [6]:
# Define a class to avoid cluttering notebook with stdout
class redirect_output(object):
    """context manager for reditrecting stdout/err to files"""


    def __init__(self, stdout='', stderr=''):
        self.stdout = stdout
        self.stderr = stderr

    def __enter__(self):
        self.sys_stdout = sys.stdout
        self.sys_stderr = sys.stderr

        if self.stdout:
            sys.stdout = open(self.stdout, 'w')
        if self.stderr:
            if self.stderr == self.stdout:
                sys.stderr = sys.stdout
            else:
                sys.stderr = open(self.stderr, 'w')

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self.sys_stdout
        sys.stderr = self.sys_stderr

# Basic demo of Audit class

In [27]:
audit_type = "ATHENA"
alpha = 0.1
delta = 1.0
candidates = ["A", "B"]
results = [600, 300]
ballots_cast = 1000
winners = 1
name = "test_election"
model = "bin"
pstop_goal = [.5, .7, .9]
round_schedule = []

In [8]:
a = make_audit(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule)

In [29]:
x = a.find_next_round_size(pstop_goal)

setting round schedule


A (600) vs B (300)
	margin:	0.3333333333333333
	pstop goals: [0.5, 0.7, 0.9]
	scaled round schedule: [45]
		0.5:	[50, 89]	0.9084557253241831
		0.7:	[50, 106]	0.9472890714209767
		0.9:	[50, 146]	0.9819758295831421


In [30]:
x

{'detailed': {'A-B': {'pstop_goal': [0.5, 0.7, 0.9],
   'next_round_sizes': [89, 106, 146],
   'prob_stop': [0.9084557253241831, 0.9472890714209767, 0.9819758295831421]}},
 'future_round_sizes': [89, 106, 146]}

In [31]:
sample_size = x['future_round_sizes'][0]

In [32]:
a.add_round_schedule([sample_size])

In [34]:
a.round_schedule

[89]

In [60]:
r = a.find_risk([sample_size // 2])



A (600) vs B (300)
	margin:	0.3333333333333333
{'kmins': [47], 'prob_sum': [0.9455433229072786], 'prob_tied_sum': [0.07281772720149406], 'deltas': [0.868742219891089]}
find_kmins_for_risk
{'kmins': [44], 'passed': 0}

	AUDIT result:
		observed:	[44]
		required:	[47]

		Test FAILED

Delta:		267.17895490806717
AUDIT risk:	0.5840284862127536


In [61]:
r

{'risk': 0.5840284862127536,
 'delta': 267.17895490806717,
 'passed': 0,
 'observed': [44],
 'required': [47]}

In [41]:
3 * sample_size // 2

133

In [42]:
sample_size

89

In [53]:
with redirect_output("/dev/null"):
    out = [(w, a.find_risk([w])['risk']) for w in range(0, sample_size, 10)]
out

[(0, 1.0000000000000067),
 (10, 1.0000000000000056),
 (20, 0.9999999752125589),
 (30, 0.9993323688712253),
 (40, 0.8554588180094498),
 (50, 0.14673318555641768),
 (60, 0.0013623133744939024),
 (70, 2.676254170296155e-06),
 (80, 3.426742231597338e-09)]

In [56]:
r = a.find_risk([sample_size])



A (600) vs B (300)
	margin:	0.3333333333333333
{'kmins': [47], 'prob_sum': [0.9455433229072786], 'prob_tied_sum': [0.07281772720149406], 'deltas': [0.868742219891089]}
find_kmins_for_risk
{'kmins': [89], 'passed': 1}

	AUDIT result:
		observed:	[89]
		required:	[47]

		Test passed

Delta:		7.593682622316245e-12
AUDIT risk:	7.593682622316245e-12


In [13]:
r

{'risk': 0.11591596559800159,
 'delta': 0.5938312972095313,
 'passed': 1,
 'observed': [30],
 'required': [28]}

In [62]:
x = find_next_round_size(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule)

setting round schedule


A (600) vs B (300)
	margin:	0.3333333333333333
	pstop goals: [0.5, 0.7, 0.9]
	scaled round schedule: []
		0.5:	[34]	0.5847595987710061
		0.7:	[48]	0.7610118463711638
		0.9:	[74]	0.9227167435192974


In [63]:
x

{'detailed': {'A-B': {'pstop_goal': [0.5, 0.7, 0.9],
   'next_round_sizes': [34, 48, 74],
   'prob_stop': [0.5847595987710061, 0.7610118463711638, 0.9227167435192974]}},
 'future_round_sizes': [34, 48, 74]}

In [76]:
round_schedule = a.round_schedule

In [77]:
round_schedule

[89]

In [87]:
a.round_schedule

[89]

In [88]:
a.add_round_schedule(round_schedule)

In [89]:
a.round_schedule

[89]

In [90]:
x = a.find_next_round_size(pstop_goal)

setting round schedule


A (600) vs B (300)
	margin:	0.3333333333333333
	pstop goals: [0.5, 0.7, 0.9]
	scaled round schedule: [80]
FULL RECOUNT is suggested!
Probability of stopping at: [80, 180] is 0.8755596079882674
		0.5:	[89, 145]	0.9760422310736788
		0.7:	[89, 167]	0.9862358931179325
		0.9:	[89, 200]	0.8755596079882674


In [91]:
x

{'detailed': {'A-B': {'pstop_goal': [0.5, 0.7, 0.9],
   'next_round_sizes': [145, 167, 200],
   'prob_stop': [0.9760422310736788, 0.9862358931179325, 0.8755596079882674]}},
 'future_round_sizes': [145, 167, 200]}

In [80]:
x

{'detailed': {'A-B': {'pstop_goal': [0.5, 0.7, 0.9],
   'next_round_sizes': [34, 48, 74],
   'prob_stop': [0.5847595987710061, 0.7610118463711638, 0.9227167435192974]}},
 'future_round_sizes': [34, 48, 74]}

In [92]:
below_kmin = max(r['required']) - max(r['observed'])

In [93]:
(max(r['required']), max(r['observed']))

(47, 44)

In [94]:
future_round_sizes = x['future_round_sizes']

In [95]:
below_kmin

3

In [96]:
future_round_sizes

[145, 167, 200]

In [97]:
list(map(lambda x: x - max(round_schedule) + 2 * below_kmin, future_round_sizes))

[62, 84, 117]

# Try to reproduce R2B2/Athena vs BRAVO
Sample Sizes for 90% probability of ending a Ballot Polling Audit of 2016 statewide Presidential contest

with risk limit 0.1, larger margins


In [16]:
# Read in data from 2016
election_2016 = json.load(open('data/2016_election.json'))

In [17]:
election_2016['Alabama']

{'contests': {'presidential': {'winners': 1,
   'candidates': ['Clinton', 'Trump'],
   'results': [729547, 1318255],
   'ballots_cast': 2123372,
   'state_id': 1,
   'margin': -0.2874828718792149}}}

In [18]:
def sample_state(state):
    "Return sample information for given state from 2016"

    election = election_2016[state]
    candidates = election['contests']['presidential']['candidates']
    results = election['contests']['presidential']['results']
    ballots_cast = election['contests']['presidential']['ballots_cast']
    athena_sample = sample90v(results[0], results[1], ballots_cast)
    return athena_sample

In [19]:
states = ['Alabama', 'Maryland', 'New York', 'Rhode Island', 'New Jersey', 'Ohio', 'Virginia',
          'Georgia', 'North Carolina', 'Arizona', 'Nevada' ]
# skip 'Minnesota', 'Florida', 'Wisconsin', 'Pennsylvania', 'Michigan']

In [20]:
athena_results = {}
with redirect_output("debug_output.txt"):
  for state in states:
    athena_results[state] = sample_state(state)

In [21]:
{s: r[0]  for s, r in athena_results.items()}

{'Alabama': 94,
 'Maryland': 98,
 'New York': 140,
 'Rhode Island': 280,
 'New Jersey': 350,
 'Ohio': 1018,
 'Virginia': 2329,
 'Georgia': 2567,
 'North Carolina': 4964,
 'Arizona': 5150,
 'Nevada': 10675}

# Misc snippets of code

In [22]:
with redirect_output("my_output.txt"):
    x = sample_state('Alabama')

In [23]:
x

(94,
 {'detailed': {'A-B': {'pstop_goal': [0.9],
    'next_round_sizes': [94],
    'prob_stop': [0.9204297020802105]}},
  'future_round_sizes': [94]})

In [24]:
e3 = find_next_round_size(audit_type, alpha, delta, ["A", "B", "C"], [600, 300, 100], ballots_cast, winners, name, model, pstop_goal, round_schedule)

setting round schedule


A (600) vs B (300)
	margin:	0.3333333333333333
	pstop goals: [0.7, 0.8, 0.9]
	scaled round schedule: []
		0.7:	[48]	0.7610118463711638
		0.8:	[55]	0.8317821224884329
		0.9:	[74]	0.9227167435192974


A (600) vs C (100)
	margin:	0.7142857142857143
	pstop goals: [0.7, 0.8, 0.9]
	scaled round schedule: []
		0.7:	[15]	0.8383950694384561
		0.8:	[15]	0.8383950694384561
		0.9:	[20]	0.961167305496754


In [25]:
e3

{'detailed': {'A-B': {'pstop_goal': [0.7, 0.8, 0.9],
   'next_round_sizes': [48, 55, 74],
   'prob_stop': [0.7610118463711638, 0.8317821224884329, 0.9227167435192974]},
  'A-C': {'pstop_goal': [0.7, 0.8, 0.9],
   'next_round_sizes': [15, 15, 20],
   'prob_stop': [0.8383950694384561, 0.8383950694384561, 0.961167305496754]}},
 'future_round_sizes': [48, 55, 74]}

In [26]:
election = {
    "alpha": alpha,
    "delta": delta,
    "candidates": candidates,
    "results": results,
    "ballots_cast": ballots_cast,
    "winners": winners,
    "name": name,
    "model": model,
    "pstop": pstop_goal,
    "round_schedule": round_schedule,
    }